In [5]:
import pandas as pd
dados = pd.read_csv('Admission_Predict_Ver1.1.csv')

In [6]:
dados.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [4]:
dados.tail()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
495,496,332,108,5,4.5,4.0,9.02,1,0.87
496,497,337,117,5,5.0,5.0,9.87,1,0.96
497,498,330,120,5,4.5,5.0,9.56,1,0.93
498,499,312,103,4,4.0,5.0,8.43,0,0.73
499,500,327,113,4,4.5,4.5,9.04,0,0.84


In [7]:
dados.drop('Serial No.',axis=1,inplace=True)

In [8]:
dados.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [10]:
previsores = dados.drop('Chance of Admit ',axis=1)
classe = dados['Chance of Admit ']

In [31]:
#Separando os dados para teste e treino
from sklearn.model_selection import train_test_split 
x_treino,x_teste,y_treino,y_teste = train_test_split(previsores,classe, test_size=0.25, random_state=45)

In [32]:
def modelos_regressores(a,b,c,d):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import ElasticNet
    
    x_treino = a
    y_treino = b  
    x_teste = c
    y_teste = d  
    linear = LinearRegression()
    ridge = Ridge()
    lasso = Lasso()
    elastic = ElasticNet()
    
    linear.fit(x_treino,y_treino)
    ridge.fit(x_treino,y_treino)
    lasso.fit(x_treino,y_treino)
    elastic.fit(x_treino,y_treino)
    
    print(f"""
    Regressão Linear = {linear.score(x_teste,y_teste)}
    Regressão Ridge = {ridge.score(x_teste,y_teste)}
    Regressão Lasso = {lasso.score(x_teste,y_teste)}
    Regressão ElasticNet = {elastic.score(x_teste,y_teste)}
    """)

In [33]:
modelos_regressores(x_treino,y_treino,x_teste,y_teste)


    Regressão Linear = 0.842323882631776
    Regressão Ridge = 0.8423277438321499
    Regressão Lasso = 0.24217659796383995
    Regressão ElasticNet = 0.5595492268157068
    


In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

modelo = LinearRegression()
kfold = KFold(n_splits=5)
resultado = cross_val_score(modelo,previsores,classe,cv=kfold)

In [12]:
print(resultado)

[0.67763918 0.79424809 0.86447645 0.81935698 0.89828691]


In [13]:
print(resultado.mean())

0.8108015220863883


In [25]:
def modelos_regressores_kfold(a,b):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import KFold
    
    kfold = KFold(n_splits=10)
    x = a
    y = b
    linear = LinearRegression()
    ridge = Ridge()
    lasso = Lasso()
    elastic = ElasticNet()
    
    resultado_linear = cross_val_score(linear, x, y, cv= kfold)
    resultado_ridge = cross_val_score(ridge, x, y, cv= kfold)
    resultado_lasso = cross_val_score(lasso, x, y, cv= kfold)
    resultado_elastic = cross_val_score(elastic, x, y, cv= kfold)
    dic_modelos = {'Linear': resultado_linear.mean(),'Ridge': resultado_ridge.mean(),'Lasso': resultado_lasso.mean(),'ElasticNet': resultado_elastic.mean()}
    melhor_modelo = max(dic_modelos, key=dic_modelos.get)
    
    print(f"""
    Regressão Linear = {resultado_linear.mean()}
    Regressão Ridge = {resultado_ridge.mean()}
    Regressão Lasso = {resultado_lasso.mean()}
    Regressão ElasticNet = {resultado_elastic.mean()}
    ------------------------------------------
    Melhor modelo: {melhor_modelo}
    Valor do modelo: {dic_modelos[melhor_modelo]}
    """)

In [26]:
modelos_regressores_kfold(previsores,classe)


    Regressão Linear = 0.8131223770253244
    Regressão Ridge = 0.813034910244318
    Regressão Lasso = 0.2148164268942911
    Regressão ElasticNet = 0.5352550870169864
    ------------------------------------------
    Melhor modelo: Linear
    Valor do modelo: 0.8131223770253244
    


In [33]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNet

#Definindo os valores que serão testados:
valores = {'alpha': [0.1,0.5,1,2,5,10,25,50,100,150,200,300,500,750,1000,1500,2000,3000,5000],'l1_ratio':[0.02,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}

#Criando o modelo:
modelo = ElasticNet()
procura = RandomizedSearchCV(estimator = modelo, param_distributions=valores , n_iter=150, cv=5, random_state=15)
procura.fit(previsores,classe)

#Imprimmindo o resultado:
print(f'Melhor score: {procura.best_score_}')
print(f'Melhor alpha: {procura.best_estimator_.alpha}')
print(f'Melhor score: {procura.best_estimator_.l1_ratio}')

Melhor score: 0.7819021542408088
Melhor alpha: 0.1
Melhor score: 0.02


In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

#Definindo os valores que serão testados:
valores = {'alpha':[0.1,0.5,1,2,5,10,25,50,100],'l1_ratio':[0.02,0.05,0.1,0.2,0.3,0.4,0.5,0.8]}

#Criando o modelo:
modelo = ElasticNet()
procura = GridSearchCV(estimator = modelo, param_grid = valores, cv = 5)
procura.fit(previsores,classe)

#Imprimmindo o resultado:
print(f'Melhor score: {procura.best_score_}')
print(f'Melhor alpha: {procura.best_estimator_.alpha}')
print(f'Melhor score: {procura.best_estimator_.l1_ratio}')

Melhor score: 0.7819021542408088
Melhor alpha: 0.1
Melhor score: 0.02
